In [10]:
import os
import shutil
from pprint import pprint
from tqdm import tqdm

In [28]:
basepath = os.path.normpath(
    os.path.join(os.getcwd(), "..", "datasets/OCTA-500/OCTA500_MONAI_3mm")
)
paths = sorted(os.listdir(basepath))
paths = [path for path in paths if path != '.ipynb_checkpoints']
all_samples = []
for path in paths:
    all_samples.append(
        {
            "vol_OCT": os.path.join(basepath, path, "vol_OCT.nii.gz"),
            "vol_OCTA": os.path.join(basepath, path, "vol_OCTA.nii.gz"),
            "label_3D_FAZ": os.path.join(basepath, path, "label_3D_FAZ.nii.gz"),
            "label_3D_Layers": os.path.join(basepath, path, "label_3D_Layers.nii.gz"),
        }
    )
pprint(all_samples[0])

{'label_3D_FAZ': '/workspace/Projects/datasets/OCTA-500/OCTA500_MONAI_3mm/10301/label_3D_FAZ.nii.gz',
 'label_3D_Layers': '/workspace/Projects/datasets/OCTA-500/OCTA500_MONAI_3mm/10301/label_3D_Layers.nii.gz',
 'vol_OCT': '/workspace/Projects/datasets/OCTA-500/OCTA500_MONAI_3mm/10301/vol_OCT.nii.gz',
 'vol_OCTA': '/workspace/Projects/datasets/OCTA-500/OCTA500_MONAI_3mm/10301/vol_OCTA.nii.gz'}


### Prepare target directories for labels 
**Groundtruth:** indicates volumes that already have been "manually segmented"

**Backup:** Backup of all groundtruth segmentation, to allow simulating incremental batches of manual annotations 

In [23]:
tgt_path_images = "/workspace/Projects/datasets/OCTA-500/OCTA500_MONAILABEL_3mm"

tgt_path_labels = os.path.join(tgt_path_images, "labels", "final")
if not os.path.exists(tgt_path_labels):
    os.makedirs(tgt_path_labels)

tgt_path_labels_backup = os.path.join(tgt_path_images, "labels", "backup")
if not os.path.exists(tgt_path_labels_backup):
    os.makedirs(tgt_path_labels_backup)

### Copy files

In [30]:
num_segs = 100
verbose = False
for idx, sample in enumerate(tqdm(all_samples)):
    src_img_file = sample["vol_OCT"]
    subject_id = os.path.split(os.path.split(src_img_file)[0])[1]
    tgt_filename = os.path.split(src_img_file)[1].replace(".nii.gz", f"_{subject_id}.nii.gz")
    
    # copy image
    tgt_img_file = os.path.join(tgt_path_images, tgt_filename)
    if not os.path.exists(tgt_img_file):
        shutil.copyfile(src_img_file, tgt_img_file)
    
    # copy gt labels
    src_lab_file = sample["label_3D_Layers"]
    if idx in list(range(num_segs)):
        tgt_lab_file = os.path.join(tgt_path_labels, tgt_filename)
        if not os.path.exists(tgt_lab_file):
            shutil.copyfile(src_lab_file, tgt_lab_file)
    
    # copy backup labels
    tgt_lab_file = os.path.join(tgt_path_labels_backup, tgt_filename)
    if not os.path.exists(tgt_lab_file):
        shutil.copyfile(src_lab_file, tgt_lab_file)
    
    if verbose:
        print("##################")
        print("Image file:")
        print(f"From:{src_img_file}\nTo: {tgt_img_file}")
        print("\nLabelfile:")
        print(f"From:{src_lab_file}\nTo: {tgt_lab_file}")
    #if idx>15:
    #    break

100%|██████████| 200/200 [00:00<00:00, 3971.01it/s]
